# SignXAI2 TensorFlow - Time Series (ECG) Analysis

ECG analysis with TensorFlow using dynamic method parsing.

## Features:
- 12-lead ECG support
- Dynamic parameters
- Real-time visualization

In [ ]:
import tensorflow as tf
import numpy as np
from signxai.api import explain
from signxai.utils.utils import remove_softmax
from utils.ecg_data import load_and_preprocess_ecg
from utils.ecg_visualization import plot_ecg

In [ ]:
# ECG-specific methods with dynamic parameters
ecg_methods = [
    'gradient',
    'gradient_x_input_x_sign_mu_neg_0_5',
    'smoothgrad_noise_0_3_samples_50',
    'integrated_gradients_steps_100'
]

# Process ECG data
for method in ecg_methods:
    explanation = explain(model, ecg_data, method_name=method)
    plot_ecg(ecg_data, explanation, title=f'TensorFlow: {method}')